In [4]:
# cree une spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Streaming from spring-boot") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.sql.shuffle.partitions", 4)
    .master("local[*]") 
    .getOrCreate()
)

spark

In [5]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "ed-kafka:29092")
    .option("subscribe", "logsTopic")
    .option("startingOffsets", "earliest")
    .load()
)


In [6]:
# View schema for raw kafka_df
kafka_df.printSchema()
#kafka_df.show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql.functions import expr

kafka_logs_df = kafka_df.withColumn("value", expr("cast(value as string)"))

In [9]:
from pyspark.sql.functions import regexp_extract, col

# Define the regex pattern
log_pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) \[([A-Z]+)] ([^\s]+) ([^:]+): ([^-]+) - (.+)"

# Extract fields using the regex pattern
parsed_kafka_logs_df = kafka_logs_df.withColumn("timestamp", regexp_extract(col("value"), log_pattern, 1)) \
    .withColumn("logLevel", regexp_extract(col("value"), log_pattern, 2)) \
    .withColumn("threadName", regexp_extract(col("value"), log_pattern, 3)) \
    .withColumn("loggerName", regexp_extract(col("value"), log_pattern, 4)) \
    .withColumn("message", regexp_extract(col("value"), log_pattern, 5)) \
    .withColumn("contextData", regexp_extract(col("value"), log_pattern, 6))



In [10]:
# Check the schema of the flattened_df, place a sample json file and change readStream to read 
parsed_kafka_logs_df.printSchema()
#flattened_df.show(truncate=False)

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- logLevel: string (nullable = true)
 |-- threadName: string (nullable = true)
 |-- loggerName: string (nullable = true)
 |-- message: string (nullable = true)
 |-- contextData: string (nullable = true)



In [ ]:
# Write the output to console sink to check the output

(parsed_kafka_logs_df
 .writeStream
 .format("console")
 .outputMode("append")
 .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())